In [1]:
import sys
import os
# when python interpreter is different, add path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import matplotlib.pyplot as plt

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 
from collections import defaultdict

from pathlib import Path
import time
import json
from json import JSONEncoder

# Special functions and optimizations
from typing import Callable, Union, Tuple
from scipy.spatial.distance import cdist  # For space and time distance
from scipy.special import gamma, kv  # Bessel function and gamma function
from scipy.interpolate import splrep, splev

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchcubicspline import natural_cubic_spline_coeffs, NaturalCubicSpline

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data

In [8]:
lat_lon_resolution = [10,10]
years = ['2024']
month_range =[7,8]
nheads = 200

# input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates"
input_path = "C:\\Users\\joonw\\tco\\GEMS_TCO-2\\Exercises\\st_model\\estimates\\"   # window
input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates/save/Apirl_16_24smooth_0.5"   # mac
# input_filename = "vecc_extra_estimates_50_july24.pkl"
# input_filename = "vecc_inter_estimates_1250_july24.pkl"

# input_filename = "vecc_inter_estimates_5000_july24.pkl"
# input_filename = "estimation_200_july24.pkl"
input_filename = "full_estimates_1250_july24.pkl"
input_filepath = os.path.join(input_path, input_filename)
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    amarel_map1250= pickle.load(pickle_file)

# Assuming df_1250 is your DataFrame
df_1250 = pd.DataFrame()
for key in amarel_map1250:
    tmp = pd.DataFrame(amarel_map1250[key][0].reshape(1, -1), columns=['sigmasq', 'range_lat', 'range_lon', 'advec_lat', 'advec_lon', 'beta', 'nugget'])
    tmp['loss'] = amarel_map1250[key][1]
    df_1250 = pd.concat((df_1250, tmp), axis=0)

# Generate date range
date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(df_1250):
    df_1250.index = date_range
else:
    print("The number of dates does not match the number of rows in the DataFrame.")

df = df_1250
# Save DataFrame to CSV
output_filename = 'full_estimates_1250_july24.csv'
output_csv_path = os.path.join(input_path, output_filename)
df.to_csv(output_csv_path, index=False)
for day in range(12,13):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    # parameters
    mm_cond_number = 10 
    idx_for_datamap= [ 8*(day),8*(day+1)]
    # params = [ 27.25, 2.18, 2.294, 4.099e-4, -0.07915, 0.0999, 3.65]   #200
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    # data
    # input_path = Path("C:\\Users\\joonw\\tco\\Extracted_data")  # window
    input_path = Path("/Users/joonwonlee/Documents/GEMS_DATA")
    instance = load_data(input_path)
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)


 Day 12 data size per day: 200.0 



In [9]:
spline_instance = kernels.spline(epsilon = 1e-17, coarse_factor=5, k=3, smooth = 0.5, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)
distances, non_zero_indices = spline_instance.precompute_coords_anisotropy(params, spline_instance.aggregated_data[:,:4], spline_instance.aggregated_data[:,:4])
flat_distances = distances.flatten()
spline_instance.max_distance = torch.max(distances).clone().detach()
spline_instance.max_distance_len = len(flat_distances)

spline_instance.spline_object = spline_instance.fit_cubic_spline(params)
# cov_matrix = cov_matrix.clone().detach()
full_ll = spline_instance.full_likelihood_using_spline( params, distances)
full_ll

instance_2 = kernels.vecchia_experiment(1.0, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
excat_ll = instance_2.full_likelihood(params, aggregated_data[:,:4], aggregated_data[:,2], instance_2.matern_cov_anisotropy_v05)

print(excat_ll, full_ll)

tensor(2130.5818, dtype=torch.float64, grad_fn=<MulBackward0>) tensor(2130.5819, dtype=torch.float64, grad_fn=<MulBackward0>)


In [10]:
print(params)

# optimizer, scheduler =  instance.optimizer_fun(params, lr= 0.01 , betas=(0.9, 0.99), eps=1e-8, step_size= 5, gamma=0.1)    
optimizer, scheduler = spline_instance.optimizer_fun(params, lr=0.03, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.9)  


out, epoch = spline_instance.run_full(params, optimizer,scheduler, epochs=100)


tensor([23.0360,  2.3000,  3.3470, -0.0543,  0.1150,  0.1102,  1.6049],
       dtype=torch.float64, requires_grad=True)
Epoch 1, Gradients: [ 3.82639586e-02 -1.92827436e+01 -1.59901149e+01  1.04869849e+02
 -3.59306731e+01 -5.23839481e+02 -4.87966575e+00]
 Loss: 2130.5818587448402, Parameters: [23.03603363  2.29999757  3.3469553  -0.0542808   0.11497638  0.11015477
  1.6048981 ]
Epoch 11, Gradients: [  0.33259296  -0.92272583  -6.70887273  60.62392948 -52.047816
 124.08680837   1.10535847]
 Loss: 2124.926963801744, Parameters: [22.79929384  2.49067737  3.53241376 -0.03799972  0.05550861  0.1508821
  1.50868145]
Epoch 21, Gradients: [ -0.38195001  -6.23950696  -7.69074182 -24.11834231 -11.39735428
  -4.2476962   -3.9836226 ]
 Loss: 2119.2663028113416, Parameters: [22.70140339  2.56525392  3.61445456 -0.12826187  0.11650132  0.12723796
  1.53477179]
Epoch 31, Gradients: [-3.80219061e-02  1.68876690e+00 -7.37630521e+00 -2.09568200e+01
  1.82972606e+01 -5.56132062e+01  8.17135687e-01]
 Loss

# Saved files below

In [9]:
class spline:
    def __init__(self, epsilon, coarse_factor, k, smooth):
        self.smooth = torch.tensor(smooth, dtype= torch.float64)
        self.k = k
        self.coarse_factor = coarse_factor
        self.epsilon = epsilon

    def compute_cov(self, params) :
         # fit_distances and flat_distances both 1d
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
        distances, non_zero_indices = instance_2.precompute_coords_anisotropy(params, aggregated_data[:,:4],aggregated_data[:,:4])
        
        flat_distances = distances.flatten()
        fit_distances = torch.linspace(self.epsilon, torch.max(flat_distances), len(flat_distances) // self.coarse_factor)

        # fit_distances = torch.zeros_like(distances)
        # print(fit_distances.shape)
        # Compute the covariance for non-zero distances
        non_zero_indices = fit_distances != 0
        out = torch.zeros_like(fit_distances, dtype= torch.float64)

        if torch.any(non_zero_indices):
            tmp = kv(self.smooth, torch.sqrt(fit_distances[non_zero_indices])).double().clone()
            out[non_zero_indices] = (sigmasq * (2**(1-self.smooth)) / gamma(self.smooth) *
                                    (torch.sqrt(fit_distances[non_zero_indices]) ) ** self.smooth *
                                    tmp)
        out[~non_zero_indices] = sigmasq

        # print(out.shape)
        #         
        # Compute spline coefficients
        coeffs = natural_cubic_spline_coeffs(fit_distances, out.unsqueeze(1))

        # Create spline object
        spline = NaturalCubicSpline(coeffs)
        # Interpolate using the spline
        out = spline.evaluate(distances)
        out = out.reshape(distances.shape)
        out += torch.eye(out.shape[0], dtype=torch.float64) * nugget 
        return out
     
    def full_likelihood(self,params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, cov_matrix) -> torch.Tensor:
        input_arr = input_np[:, :4]  ## input_np is aggregated data over a day.
        y_arr = y

        # Compute the covariance matrix
        # cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
        
        # Compute the log determinant of the covariance matrix
        sign, log_det = torch.slogdet(cov_matrix)
        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        
        # Extract locations
        locs = input_arr[:, :2]

        # Compute beta
        tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
        beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        mu = torch.matmul(locs, beta)
        y_mu = y_arr - mu

        # Compute the quadratic form
        quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))

        # Compute the negative log likelihood
        neg_log_lik = 0.5 * (log_det + quad_form)
     
        return  neg_log_lik
    
    def compute_full_nll(self, params, covariance_function):
        cov_mat = covariance_function(params) 
        nll = self.full_likelihood( params,aggregated_data[:,:4], aggregated_data[:,2], cov_mat)
        return nll

    def optimizer_fun(self, params, lr=0.01, betas=(0.9, 0.8), eps=1e-8, step_size=40, gamma=0.5):
        optimizer = torch.optim.Adam([params], lr=lr, betas=betas, eps=eps)
        scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)  # Decrease LR by a factor of 0.1 every 10 epochs
        return optimizer, scheduler

   # use adpating lr
    def run_full(self, params, optimizer, scheduler,  covariance_function, epochs=10 ):
        prev_loss= float('inf')

        tol = 1e-4  # Convergence tolerance
        for epoch in range(epochs):  # Number of epochs
            optimizer.zero_grad()  # Zero the gradients 
            
            loss = self.compute_full_nll(params, covariance_function)
            loss.backward()  # Backpropagate the loss
            
            # Print gradients and parameters every 10th epoch
            if epoch % 10 == 0:
                print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
            
            # if epoch % 500 == 0:
            #     print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
            
            optimizer.step()  # Update the parameters
            scheduler.step()  # Update the learning rate
            # Check for convergence
            if abs(prev_loss - loss.item()) < tol:
                print(f"Converged at epoch {epoch}")
                print(f'Epoch {epoch+1}, : Loss: {loss.item()}, \n vecc Parameters: {params.detach().numpy()}')
                break

            prev_loss = loss.item()
        print(f'FINAL STATE: Epoch {epoch+1}, Loss: {loss.item()}, \n vecc Parameters: {params.detach().numpy()}')
        return params.detach().numpy().tolist() + [ loss.item()], epoch



test

# Train a model

In [ ]:
print(params)

instance_2 = kernels.vecchia_experiment(1.0, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
instance = spline( epsilon = 1e-8, coarse_factor = 4, k=3, smooth= 0.5)
# optimizer, scheduler =  instance.optimizer_fun(params, lr= 0.01 , betas=(0.9, 0.99), eps=1e-8, step_size= 5, gamma=0.1)    
optimizer, scheduler = instance.optimizer_fun(params, lr=0.03, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.9)  
out, epoch = instance.run_full(params, optimizer,scheduler, instance.compute_cov, epochs=100)


In [16]:
out1 = splinenn.evaluate(distances)
out1 = out1.reshape(distances.shape)

distances

tensor([[0.0000, 2.8198, 2.0805,  ..., 1.4813, 0.5896, 1.7424],
        [2.8198, 0.0000, 9.7185,  ..., 3.6351, 3.4548, 4.7177],
        [2.0805, 9.7185, 0.0000,  ..., 4.4068, 2.6395, 4.0108],
        ...,
        [1.4813, 3.6351, 4.4068,  ..., 0.0000, 0.8485, 0.0821],
        [0.5896, 3.4548, 2.6395,  ..., 0.8485, 0.0000, 1.0949],
        [1.7424, 4.7177, 4.0108,  ..., 0.0821, 1.0949, 0.0000]],
       dtype=torch.float64, grad_fn=<AddBackward0>)

In [17]:
smooth = 0.5

instance_2 = kernels.vecchia_experiment(smooth, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
instance = spline( epsilon = 1e-15, coarse_factor = 2, k=3, smooth= smooth)

distances, non_zero_indices = instance_2.precompute_coords_anisotropy(params, aggregated_data[:,:4],aggregated_data[:,:4])

flat_distances = distances.flatten()
sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
epsilon = 1e-8
coarse_factor = 4

fit_distances = torch.linspace(epsilon, torch.max(flat_distances), len(flat_distances) // coarse_factor)
print(fit_distances.shape)
# Compute the covariance for non-zero distances
non_zero_indices = fit_distances != 0
out = torch.zeros_like(fit_distances, dtype= torch.float64)

if torch.any(non_zero_indices):
    tmp = kv(smooth, torch.sqrt(fit_distances[non_zero_indices])).double().clone()
    out[non_zero_indices] = (sigmasq * (2**(1-smooth)) / gamma(smooth) *
                            (torch.sqrt(fit_distances[non_zero_indices]) ) ** smooth *
                            tmp)
    
out[~non_zero_indices] = sigmasq

print(out.shape)

# Compute spline coefficients
coeffs = natural_cubic_spline_coeffs(fit_distances, out.unsqueeze(1))

# Create spline object
splinenn = NaturalCubicSpline(coeffs)

# Interpolate using the spline
out1 = splinenn.evaluate(distances)
out1 = out1.reshape(distances.shape)
out1 += torch.eye(out1.shape[0], dtype=torch.float64) * nugget 

print(out1)
out2 = instance_2.matern_cov_anisotropy_kv(params, aggregated_data[:,:4],aggregated_data[:,:4])


print(out2)
instance.full_likelihood( params,aggregated_data[:,:4], aggregated_data[:,2], out1)


torch.Size([40000])
torch.Size([40000])
tensor([[24.8247,  4.3205,  5.4749,  ...,  6.8587, 10.7482,  6.1878],
        [ 4.3205, 24.8247,  1.0255,  ...,  3.4418,  3.6106,  2.6395],
        [ 5.4749,  1.0255, 24.8247,  ...,  2.8387,  4.5628,  3.1264],
        ...,
        [ 6.8587,  3.4418,  2.8387,  ..., 24.8247,  9.2207, 17.3917],
        [10.7482,  3.6106,  4.5628,  ...,  9.2207, 24.8247,  8.1353],
        [ 6.1878,  2.6395,  3.1264,  ..., 17.3917,  8.1353, 24.8247]],
       dtype=torch.float64, grad_fn=<AsStridedBackward0>)
tensor([[24.8270,  4.3205,  5.4749,  ...,  6.8587, 10.7482,  6.1878],
        [ 4.3205, 24.8270,  1.0255,  ...,  3.4418,  3.6106,  2.6395],
        [ 5.4749,  1.0255, 24.8270,  ...,  2.8387,  4.5628,  3.1264],
        ...,
        [ 6.8587,  3.4418,  2.8387,  ..., 24.8270,  9.2207, 17.3917],
        [10.7482,  3.6106,  4.5628,  ...,  9.2207, 24.8270,  8.1353],
        [ 6.1878,  2.6395,  3.1264,  ..., 17.3917,  8.1353, 24.8270]],
       dtype=torch.float64, grad_f

tensor(601.6160, dtype=torch.float64, grad_fn=<MulBackward0>)